# LINGER ANALYSIS

In this notebook, we present a GNR inference, largely based on the works of:
> [Yuan, Qiuyue, and Zhana Duren. "Inferring gene regulatory networks from single-cell multiome data using atlas-scale external data." Nature Biotechnology (2024): 1-11.](https://doi.org/10.1038/s41587-024-02182-7)

and the tutorial available at [LINGER's Github](https://github.com/Durenlab/LINGER/blob/main/docs/PBMC.md).

The data used are output files from ***PBMC from a Healthy Donor - Granulocytes Removed Through Cell Sorting (10k)*** *Single Cell Multiome ATAC + Gene Expression experiment* on *10X Chromium* platform analysed with Cell Ranger ARC, which are available on [10XGenomics website](https://www.10xgenomics.com/datasets/pbmc-from-a-healthy-donor-granulocytes-removed-through-cell-sorting-10-k-1-standard-2-0-0). Cell type annotations were downloaded from [PBMC tutorial](https://github.com/Durenlab/LINGER/blob/main/docs/PBMC.md).

## Preparation steps

In [1]:
%matplotlib inline
import os
os.chdir('/home/kl467102/thesis/')

In [2]:
import scanpy as sc
import scipy
import pandas as pd
import time
import os

In [3]:
input_dir = 'data/filtered_feature_bc_matrix' # path to 10x output files
label_file = 'data/PBMC_label.txt' # path to cell-type annotations

The input consists of `matrix.mtx`, `features.tsv` and `barcodes.tsv` and an annotation file.

In [5]:
matrix=scipy.io.mmread(os.path.join(input_dir, 'matrix.mtx'))
features=pd.read_csv(os.path.join(input_dir, 'features.tsv'), sep='\t', header=None)
barcodes=pd.read_csv(os.path.join(input_dir, 'barcodes.tsv'), sep='\t', header=None)
label=pd.read_csv(label_file, sep='\t', header=0)

We have loaded the data, now we can inspect them.

In [6]:
matrix.shape

(180488, 11898)

In [7]:
pd.concat([features.head(2), features.tail(2)])

,0,1,2,3,4,5
0,ENSG00000243485,MIR1302-2HG,Gene Expression,chr1,29553,30267
1,ENSG00000237613,FAM138A,Gene Expression,chr1,36080,36081
180486,KI270713.1:31340-32243,KI270713.1:31340-32243,Peaks,KI270713.1,31340,32243
180487,KI270713.1:36927-37836,KI270713.1:36927-37836,Peaks,KI270713.1,36927,37836


In [8]:
barcodes.head(3)

,0
0,AAACAGCCAAGGAATC-1
1,AAACAGCCAATCCCTT-1
2,AAACAGCCAATGCGCT-1


In [9]:
label.head(3)

,barcode_use,label
0,AAACAGCCAAGGAATC-1,naive CD4 T cells
1,AAACAGCCAATCCCTT-1,memory CD4 T cells
2,AAACAGCCAATGCGCT-1,naive CD4 T cells


## Step 1 - preprocessing

### Extraction of matrices for RNASeq and ATACSeq

In [11]:
from LingerGRN.preprocess import *
adata_RNA, adata_ATAC = get_adata(matrix, features, barcodes, label)

/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/LingerGRN/preprocess.py:278: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_RNA.obs['label']=label.loc[adata_RNA.obs['barcode']]['label'].values
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/LingerGRN/preprocess.py:281: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_ATAC.obs['label']=label.loc[adata_ATAC.obs['barcode']]['label'].values


### Removing low count cells

In [12]:
sc.pp.filter_cells(adata_RNA, min_genes=200)
sc.pp.filter_genes(adata_RNA, min_cells=3)
sc.pp.filter_cells(adata_ATAC, min_genes=200)
sc.pp.filter_genes(adata_ATAC, min_cells=3)

/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


In [13]:
# Find common barcodes between RNA and ATAC datasets
selected_barcode = list(set(adata_RNA.obs['barcode']) & set(adata_ATAC.obs['barcode']))

# Filter both datasets to keep only the shared barcodes
adata_RNA = adata_RNA[adata_RNA.obs['barcode'].isin(selected_barcode)]
adata_ATAC = adata_ATAC[adata_ATAC.obs['barcode'].isin(selected_barcode)]

### Generating metacells

Metacells are generated to create more stable, representative profiles of groups of cells. 

In [14]:
from LingerGRN.pseudo_bulk import *

In [15]:
samplelist=list(set(adata_ATAC.obs['sample'].values)) 
tempsample=samplelist[0]
TG_pseudobulk=pd.DataFrame([])
RE_pseudobulk=pd.DataFrame([])

In [16]:
singlepseudobulk = (adata_RNA.obs['sample'].unique().shape[0]*adata_RNA.obs['sample'].unique().shape[0]>100)
for tempsample in samplelist:
    adata_RNAtemp=adata_RNA[adata_RNA.obs['sample']==tempsample]
    adata_ATACtemp=adata_ATAC[adata_ATAC.obs['sample']==tempsample]
    TG_pseudobulk_temp,RE_pseudobulk_temp=pseudo_bulk(adata_RNAtemp,adata_ATACtemp,singlepseudobulk)                
    TG_pseudobulk=pd.concat([TG_pseudobulk, TG_pseudobulk_temp], axis=1)
    RE_pseudobulk=pd.concat([RE_pseudobulk, RE_pseudobulk_temp], axis=1)
    RE_pseudobulk[RE_pseudobulk > 100] = 100

/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:842: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:372: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:842: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


Write preprocessed data to a file:

In [17]:
linger_storage = 'results/linger_storage'
os.makedirs(linger_storage, exist_ok = True)

adata_ATAC.write(os.path.join(linger_storage,'adata_ATAC.h5ad'))
adata_RNA.write(os.path.join(linger_storage,'adata_RNA.h5ad'))
TG_pseudobulk=TG_pseudobulk.fillna(0)
RE_pseudobulk=RE_pseudobulk.fillna(0)
pd.DataFrame(adata_ATAC.var['gene_ids']).to_csv('data/Peaks.txt',header=None,index=None) #hard coded path to Peaks in preprocess
TG_pseudobulk.to_csv(os.path.join(linger_storage,'TG_pseudobulk.tsv'))
RE_pseudobulk.to_csv(os.path.join(linger_storage,'RE_pseudobulk.tsv'))

/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/kl467102/miniconda3/envs/linger_enviroment/lib/python3.10/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


## Step 2 - training

In [4]:
method = 'LINGER'
Datadir = '/home/kl467102/proj_v0/downloads/' # Directory for the downloaded general gene regulatory network for LINGER
GRNdir = Datadir+'data_bulk/'
genome = 'hg38'
outdir = os.path.join(os.getcwd(),'results/LINGER_output/')
os.makedirs(outdir, exist_ok = True)

NOTE TO SELF: Because of `pybedtools` dependency `outdir` needs to be specified as an absolute path, otherwise resulting in an error

In [22]:
from LingerGRN.preprocess import *

In [24]:
preprocess(TG_pseudobulk,RE_pseudobulk,GRNdir,genome,method,outdir)

Mapping gene expression...
Generate TF expression...
Generate RE chromatin accessibility...
Generate TF binding...


100%|████████████████████████████████████████████████████████████████████████████████| 23/23 [1:12:17<00:00, 188.60s/it]


Generate Index...


100%|█████████████████████████████████████████████████████████████████████████████| 14907/14907 [05:31<00:00, 44.97it/s]


In [5]:
import LingerGRN.LINGER_tr as LINGER_tr

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [7]:
activef='ReLU'
LINGER_tr.training(GRNdir,method,outdir,activef,'Human')

chr1


  1%|▌                                                                             | 12/1520 [09:21<19:35:47, 46.78s/it]


KeyboardInterrupt: 

## Step 3 - population GNR inference

In [ ]:
import LingerGRN.LL_net as LL_net
LL_net.TF_RE_binding(GRNdir,adata_RNA,adata_ATAC,genome,method,outdir)

In [ ]:
LL_net.cis_reg(GRNdir,adata_RNA,adata_ATAC,genome,method,outdir)

In [ ]:
LL_net.trans_reg(GRNdir,method,outdir,genome)

## Step 4 - cell-type specific GNR

In [ ]:
celltype='all'

In [ ]:
LL_net.cell_type_specific_TF_RE_binding(GRNdir,adata_RNA,adata_ATAC,genome,celltype,outdir,method)

In [ ]:
LL_net.cell_type_specific_cis_reg(GRNdir,adata_RNA,adata_ATAC,genome,celltype,outdir)

In [ ]:
LL_net.cell_type_specific_trans_reg(GRNdir,adata_RNA,celltype,outdir)

# Session information

In [27]:
import session_info
session_info.show()